In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? n
Nothing done.


In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
from __future__ import division

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import sys
sys.path.insert(0, '/home/knight/matar/PYTHON/ECOGpy/pacpy')
#sys.path.insert(0, '/home/knight/matar/PYTHON/ECOGpy/ecogtools')
sys.path.insert(0, '/home/knight/matar/PYTHON/ECOGpy/')


In [6]:
import os
import pandas as pd
from utils.loadmat import loadmat
from glob import glob
from collections import defaultdict

In [7]:
base = '/home/knight/matar/MATLAB/DATA/Avgusta/'


## figure out dropped clusters

In [42]:
filename = os.path.join(base, 'PCA', 'duration_dict_500_FINAL', 'groupidx_activeclusters_hclust_withduration_maxRTlocked_withcriteria_labeled_ROI.csv')
df = pd.read_csv(filename)
#print df.groupby(['subj','task','group']).count().shape #177 analyzed clusters including subsequently dropped subjects
df = df[df.subj.apply(lambda x: x not in ['GP44','ST28','GP27'])]
df[(df.pattern == 'S+sustained') | (df.pattern == 'sustained')].shape
df.groupby(['subj','task','group']).count().shape 
#151 analyzed clusters without GP44, ST28 and GP27


(151, 24)

In [43]:
#cluster and pattern designations
filename = os.path.join(base, 'PCA', 'csvs_FINAL', 'old', 'single_electrode_windows_withdesignation_EDITED.csv')
df = pd.read_csv(filename)
df = df[df.subj.apply(lambda x: x not in ['GP44','ST28','GP27'])]
print df.groupby(['subj','task','cluster']).count().shape 

(151, 9)


In [48]:
filename = os.path.join(base, 'PCA', 'hclust', 'groupidx_activeclusters_hclust_thresh15_forbothstimandresp.csv')
df = pd.read_csv(filename)
df = df[df.subj.apply(lambda x: x not in ['GP44','ST28','GP27'])]
print df.groupby(['subj','task','group']).count().shape
df[(df['active_cluster_stim'] == True) | (df['active_cluster_resp'] == True)].groupby(['subj','task','group']).count().shape
#df[(df['active_cluster_stim'] == True)].groupby(['subj','task','group']).count().shape


(181, 6)


(153, 6)

In [45]:
filename = os.path.join(base, 'PCA', 'hclust', 'groupidx_activeclusters_hclust_thresh15stim_thresh10resp.csv')
df = pd.read_csv(filename)
df = df[df.subj.apply(lambda x: x not in ['GP44','ST28','GP27'])]
df[(df['active_cluster_stim'] == True) | (df['active_cluster_resp'] == True)].groupby(['subj','task','group']).count().shape


(152, 5)

In [46]:
filename = os.path.join(base, 'PCA', 'csvs_FINAL', 'mean_traces_all_subjs_dropSR.csv')
df = pd.read_csv(filename)

In [47]:
df.groupby(['subj','task','cluster']).count().shape #145 analyzed clusters

(145, 15)

In [ ]:
outlier_dict = defaultdict(list)
for j in df[['subj','task']].drop_duplicates().values:
    subj, task = j
    
    #load variables
    onsets_dict = {}
    subj_globals = glob(os.path.join(base, 'Subjs',subj, task, 'subj_globals*mat'))
    data_dict = loadmat(subj_globals[0])
    srate = data_dict['srate']
    
    onsetfiles = glob(os.path.join(base, 'Subjs',subj, task, 'onset*mat'))
    for f in onsetfiles:
        varname = os.path.basename(f).split('.mat')[0]
        data_dict = loadmat(f)
        onsets_dict[varname] = data_dict[varname]
        for key in onsets_dict.keys():
            if 'resp' in key:
                onsets_resp = onsets_dict[key]/srate*1000
            elif ('cue' not in key) and ('empty' not in key):
                onsets_stim = onsets_dict[key]/srate*1000
                
                
    # calculate RTs
    RTs = onsets_resp - onsets_stim;
    
    # remove outliers (mean +/- 3SDs)
    goodidx = (RTs < (np.mean(RTs) + 3 * np.std(RTs))) * (RTs > (np.mean(RTs) - 3 * np.std(RTs)))
    if not(all(goodidx)):
        outlier_dict['subj'].append(subj)
        outlier_dict['task'].append(task)
        outlier_dict['total'].append(len(RTs))
        outlier_dict['dropped'].append(sum(~goodidx))
        outlier_dict['percent'].append(sum(~goodidx)/len(RTs)*100)
        print (subj, task, (sum(~goodidx)/len(RTs)))

In [86]:
filename = '/home/knight/matar/MATLAB/DATA/Avgusta/PCA/csvs_FINAL/RTs_dropped.csv'
pd.DataFrame(outlier_dict).to_csv(filename)

In [87]:
pd.DataFrame(outlier_dict)

,dropped,percent,subj,task,total
0,1,0.800000,CP7,DecisionAud,125
1,4,1.515152,CP9,DecisionAud,264
2,1,0.869565,GP15,EmoRep,115
3,1,0.757576,GP15,FaceEmo,132
4,2,1.503759,GP15,FaceGen,133
5,1,1.333333,GP28,SelfAud,75
6,1,1.098901,GP35,EmoGen,91
7,2,2.298851,GP35,EmoRep,87
8,1,0.952381,GP35,FaceEmo,105
9,1,0.719424,JH10,EmoRep,139


defaultdict(list,
            {'dropped': [1,
              4,
              1,
              1,
              2,
              1,
              1,
              2,
              1,
              1,
              1,
              2,
              2,
              1,
              5,
              2,
              2,
              1,
              3],
             'percent': [0.0080000000000000002,
              0.015151515151515152,
              0.0086956521739130436,
              0.007575757575757576,
              0.015037593984962405,
              0.013333333333333334,
              0.01098901098901099,
              0.022988505747126436,
              0.0095238095238095247,
              0.0071942446043165471,
              0.019607843137254902,
              0.017391304347826087,
              0.014492753623188406,
              0.011235955056179775,
              0.011961722488038277,
              0.0072727272727272727,
              0.014598540145985401,
              0.0126